In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_csv("training.csv", delimiter=',')
df_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df_train.shape

(8523, 12)

In [4]:
df_train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [5]:
temp = pd.DataFrame(index=df_train.columns)
temp['data_type'] = df_train.dtypes
temp['null_count'] = df_train.isnull().sum()
temp['unique_count'] = df_train.nunique()
temp

,data_type,null_count,unique_count
Item_Identifier,object,0,1559
Item_Weight,float64,1463,415
Item_Fat_Content,object,0,5
Item_Visibility,float64,0,7880
Item_Type,object,0,16
Item_MRP,float64,0,5938
Outlet_Identifier,object,0,10
Outlet_Establishment_Year,int64,0,9
Outlet_Size,object,2410,3
Outlet_Location_Type,object,0,3


In [6]:
# imputing values for Outlet_Size

# creating a list of all those index where Outlet_Size is null
a=list(df_train[df_train['Outlet_Size'].isnull()].index)

# imputing Outlet_Type based on the Outlet_Type categories
for i in a:
    if (df_train['Outlet_Type'].iloc[i]=='Grocery Store'):
        df_train['Outlet_Size'].iloc[i]='Small'
    elif (df_train['Outlet_Type'].iloc[i]=='Supermarket Type1'):
        df_train['Outlet_Size'].iloc[i]='Small'

/Users/satwikpai/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# imputing values for Item_Weight

# this function calculates the mean of item_weights 
def fill_na(x):
    return df_train['Item_Weight'].mean()

# here we update item_weight whenever it is null
df_train['Item_Weight'] =  df_train.apply(lambda x: fill_na(x) if pd.isnull(x['Item_Weight']) else x['Item_Weight'], axis=1)

In [8]:
df_train = df_train.drop(['Item_Identifier'], axis=1)

In [9]:
# covncerting Item_Fat_Content into 'Low Fat' and 'Regular'
df_train['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'}, inplace = True)

In [10]:
temp = pd.DataFrame(index=df_train.columns)
temp['data_type'] = df_train.dtypes
temp['null_count'] = df_train.isnull().sum()
temp['unique_count'] = df_train.nunique()
temp

,data_type,null_count,unique_count
Item_Weight,float64,0,416
Item_Fat_Content,object,0,2
Item_Visibility,float64,0,7880
Item_Type,object,0,16
Item_MRP,float64,0,5938
Outlet_Identifier,object,0,10
Outlet_Establishment_Year,int64,0,9
Outlet_Size,object,0,3
Outlet_Location_Type,object,0,3
Outlet_Type,object,0,4


In [11]:
df_train = pd.get_dummies(df_train)

In [12]:
x = df_train.drop('Item_Outlet_Sales', axis=1)
y = df_train['Item_Outlet_Sales']

In [13]:
## Importing the MinMax Scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

x = pd.DataFrame(x_scaled, columns = x.columns)

In [14]:
# Importing the train test split function
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,y, random_state = 56)

## decision tree

In [185]:
# how to import decision tree regressor
from sklearn.tree import DecisionTreeRegressor

In [186]:
#creating the decision tree function
# instance creation
dt_model1 = DecisionTreeRegressor(random_state=29, max_depth=6)

In [187]:
#fitting the model
dt_model1.fit(train_x, train_y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=6,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=29, splitter='best')

In [188]:
#checking the training score
dt_model1.score(train_x, train_y)

0.6204041853553892

In [189]:
#checking the validation score
dt_model1.score(test_x, test_y)

0.5813015327549005

In [ ]:
train_accuracy = []
test_accuracy = []
for depth in range(1,20):
    dt_model = DecisionTreeRegressor(max_depth=depth, random_state=10)
    dt_model.fit(train_x, train_y)
    train_accuracy.append(dt_model.score(train_x, train_y))
    test_accuracy.append(dt_model.score(test_x, test_y))

In [ ]:
frame = pd.DataFrame({'max_depth':range(1,20), 'train_acc':train_accuracy, 'test_acc':test_accuracy})
frame.head(20)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(frame['max_depth'], frame['train_acc'], marker='o')
plt.plot(frame['max_depth'], frame['test_acc'], marker='o')
plt.xlabel('Depth of tree')
plt.ylabel('performance')
plt.legend()

# random forest

In [15]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, max_depth= 6, random_state=2)

In [16]:
rf.fit(train_x, train_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=2, verbose=0, warm_start=False)

In [17]:
pred_train_rf = rf.predict(train_x)
pred_valid_rf = rf.predict(test_x)

In [18]:
from sklearn.metrics import mean_squared_error as mse

tmp2 = mse(pred_valid_rf, test_y)

In [19]:
tmp2

1308414.2026828139

In [20]:
pred_valid_rf

array([ 371.63478777, 2490.64831222, 2999.24721099, ..., 1328.64491971,
       1546.6078968 ,  630.08240276])

In [ ]:
train_accuracy=[]
test_accuracy = []
for depth in range(1,10):
    dt_model2020 = RandomForestRegressor(min_samples_leaf=depth+1, random_state=10)
    dt_model2020.fit(train_x, train_y)
    train_accuracy.append(dt_model2020.score(train_x, train_y))
    test_accuracy.append(dt_model2020.score(test_x, test_y))

In [ ]:
frame = pd.DataFrame({'max_depth':range(1,10), 'train_acc':train_accuracy, 'test_acc':test_accuracy})
frame.head(20)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(frame['max_depth'], frame['train_acc'], marker='o')
plt.plot(frame['max_depth'], frame['test_acc'], marker='o')
plt.xlabel('Depth of tree')
plt.ylabel('performance')
plt.legend()

## ensemble 

## test set

In [21]:
df_test = pd.read_csv("test.csv", delimiter=',')
df_test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [22]:
df_test.shape

(5681, 11)

In [23]:
temp = pd.DataFrame(index=df_test.columns)
temp['data_type'] = df_test.dtypes
temp['null_count'] = df_test.isnull().sum()
temp['unique_count'] = df_test.nunique()
temp

,data_type,null_count,unique_count
Item_Identifier,object,0,1543
Item_Weight,float64,976,410
Item_Fat_Content,object,0,5
Item_Visibility,float64,0,5277
Item_Type,object,0,16
Item_MRP,float64,0,4402
Outlet_Identifier,object,0,10
Outlet_Establishment_Year,int64,0,9
Outlet_Size,object,1606,3
Outlet_Location_Type,object,0,3


In [24]:
df_test['Outlet_Size'].value_counts()

Medium    1862
Small     1592
High       621
Name: Outlet_Size, dtype: int64

In [25]:
outlet_sizetype_mode2 = pd.crosstab(df_test['Outlet_Size'],df_test['Outlet_Type'])
outlet_sizetype_mode2

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,,,,
High,0,621,0,0
Medium,0,620,618,624
Small,352,1240,0,0


In [26]:
# imputing values for Outlet_Size

# creating a list of all those index where Outlet_Size is null
a2=list(df_test[df_test['Outlet_Size'].isnull()].index)

# imputing Outlet_Type based on the Outlet_Type categories
for i in a2:
    if (df_test['Outlet_Type'].iloc[i]=='Grocery Store'):
        df_test['Outlet_Size'].iloc[i]='Small'
    elif (df_test['Outlet_Type'].iloc[i]=='Supermarket Type1'):
        df_test['Outlet_Size'].iloc[i]='Small'

/Users/satwikpai/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [27]:
# imputing values for Item_Weight

# this function calculates the mean of item_weights 
def fill_na(x):
    return df_test['Item_Weight'].mean()

# here we update item_weight whenever it is null
df_test['Item_Weight'] =  df_test.apply(lambda x: fill_na(x) if pd.isnull(x['Item_Weight']) else x['Item_Weight'], axis=1)

In [28]:
temp = pd.DataFrame(index=df_test.columns)
temp['data_type'] = df_test.dtypes
temp['null_count'] = df_test.isnull().sum()
temp['unique_count'] = df_test.nunique()
temp

,data_type,null_count,unique_count
Item_Identifier,object,0,1543
Item_Weight,float64,0,411
Item_Fat_Content,object,0,5
Item_Visibility,float64,0,5277
Item_Type,object,0,16
Item_MRP,float64,0,4402
Outlet_Identifier,object,0,10
Outlet_Establishment_Year,int64,0,9
Outlet_Size,object,0,3
Outlet_Location_Type,object,0,3


In [29]:
df_test = df_test.drop(['Item_Identifier'], axis=1)

In [30]:
# checking if there are any errors in entry of data

temp = list(df_test.columns)
for i in temp:
    print('************ Value Count in', i, '************')
    print(df_test[i].value_counts())
    print('')

************ Value Count in Item_Weight ************
12.695633    976
10.500000     57
17.600000     53
15.600000     48
20.250000     47
            ... 
7.670000       1
7.000000       1
6.320000       1
6.960000       1
8.060000       1
Name: Item_Weight, Length: 411, dtype: int64

************ Value Count in Item_Fat_Content ************
Low Fat    3396
Regular    1935
LF          206
reg          78
low fat      66
Name: Item_Fat_Content, dtype: int64

************ Value Count in Item_Visibility ************
0.000000    353
0.107493      2
0.159518      2
0.026711      2
0.056306      2
           ... 
0.053802      1
0.101626      1
0.091380      1
0.060802      1
0.050289      1
Name: Item_Visibility, Length: 5277, dtype: int64

************ Value Count in Item_Type ************
Snack Foods              789
Fruits and Vegetables    781
Household                638
Frozen Foods             570
Dairy                    454
Baking Goods             438
Canned                   435


In [31]:
# covncerting Item_Fat_Content into 'Low Fat' and 'Regular'
df_test['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'}, inplace = True)

In [32]:
df_test = pd.get_dummies(df_test)

In [33]:
## Importing the MinMax Scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(df_test)

In [34]:
df_test = pd.DataFrame(x_scaled, columns = x.columns)

In [36]:
pred1t = rf.predict(df_test)

In [37]:
pred1t

array([1611.13812961, 1333.25500534,  552.96194358, ..., 1950.41053358,
       3626.83025451, 1317.7173901 ])

In [38]:
pred2t = rf.predict(df_test)

In [39]:
pred2t

array([1611.13812961, 1333.25500534,  552.96194358, ..., 1950.41053358,
       3626.83025451, 1317.7173901 ])

In [40]:
# use simple avreage of the prediction of each row, to get final prediction for that row
from statistics import mean
final_pred = np.array([])
for i in range(0,len(df_test)):
    final_pred = np.append(final_pred, mean([pred1t[i], pred2t[i]]))

In [41]:
test2 = pd.read_csv("test.csv", delimiter=',')

In [42]:
res = pd.DataFrame ({'Item_Identifier':test2['Item_Identifier'], 'Outlet_Identifier':test2['Outlet_Identifier'], 'Item_Outlet_Sales':final_pred} )
res

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1611.138130
1,FDW14,OUT017,1333.255005
2,NCN55,OUT010,552.961944
3,FDQ58,OUT017,2535.481640
4,FDY38,OUT027,5721.315476
...,...,...,...
5676,FDB58,OUT046,2083.256744
5677,FDD47,OUT018,2600.152557
5678,NCO17,OUT045,1950.410534
5679,FDJ26,OUT017,3626.830255


In [45]:
res.to_csv('bestcheck.csv', index=False)